In [1]:
from azureml.core import Dataset, Workspace, Experiment, Datastore, Keyvault, RunConfiguration

from azureml.core.compute import ComputeTarget
from azureml.core.model import InferenceConfig
from azureml.core.resource_configuration import ResourceConfiguration
from azureml.core.webservice import AciWebservice

from azureml.pipeline.core import Pipeline, PipelineData, PipelineParameter
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep

from azureml.train.estimator import Estimator

In [2]:
ws = Workspace.from_config()
exp = Experiment(ws, "MaxFreezerTemperatureExceededPipeline", _create_in_cloud=True)
freezer_environment = ws.environments["sktime_freezer_environment"]

# Create a Pipeline

### Parameters

In [9]:
compute_target = ComputeTarget(ws, "freezertrain")
experiment = Experiment(ws, "MaxFreezerTemperatureExceededPipeline", _create_in_cloud=True)

run_config = RunConfiguration()
run_config.environment = ws.environments["sktime_freezer_environment"]

datastore = Datastore.get(ws, "sensordata")

output_df_long = PipelineData("output_df_long", datastore=ws.get_default_datastore())
output_df_nested = PipelineData("output_df_nested", datastore=ws.get_default_datastore())

time_series_length_param = PipelineParameter(name="time_series_length", default_value=10)
threshold_param = PipelineParameter(name="threshold", default_value=180.)

dataset_name_param = PipelineParameter(name="dataset_name", default_value="processed_json")

n_estimators_param = PipelineParameter(name="n_estimators", default_value=10)
train_data_split_param = PipelineParameter(name="train_data_split", default_value=.8)

redeploy_webservice_param = PipelineParameter(name="redeploy", default_value=True)
webservicename_param = PipelineParameter(name="webservice", default_value="freezerchain-prediction-v0-2")

### Build the pipeline steps

In [13]:
update_dataset_step =  PythonScriptStep(
    name="Update Dataset",
    script_name="00_update_dataset.py",
    arguments=["--dataset_name", dataset_name_param],
    compute_target=compute_target,
    source_directory="preprocess",
    runconfig=run_config,
    allow_reuse=False
)

In [14]:
first_prepro_step =  PythonScriptStep(
    name="Parse dataset",
    script_name="01_raw_to_long.py",
    arguments=["--dataset_name", dataset_name_param, "--output", output_df_long, "--time_series_length", time_series_length_param],
    outputs=[output_df_long],
    compute_target=compute_target,
    source_directory="preprocess",
    runconfig=run_config,
    allow_reuse=True
)

In [15]:
second_prepro_step = PythonScriptStep(
    name="Convert to SKTime format",
    script_name="02_long_to_nested.py",
    arguments=["--input", output_df_long, "--output", output_df_nested, "--threshold", threshold_param],
    inputs=[output_df_long],
    outputs=[output_df_nested],
    compute_target=compute_target,
    source_directory="preprocess",
    runconfig=run_config,
    allow_reuse=True
)

In [17]:
estimator_step = EstimatorStep(name="Train Model", 
    estimator=Estimator(
                        source_directory="train", 
                        entry_script="train_pipeline.py",
                        compute_target=compute_target,
                        environment_definition = freezer_environment
                        ),
    estimator_entry_script_arguments=["--input", output_df_nested, "--n_estimators", n_estimators_param, "--train_data_split", train_data_split_param],
    runconfig_pipeline_params=None, 
    inputs=[output_df_nested],
    # outputs=[model_output],
    compute_target=compute_target,
    allow_reuse=True
    )

In [18]:
deploy_step = PythonScriptStep(
    name="Deploy Model",
    script_name="deploy.py",
    arguments=["--redeploy", redeploy_webservice_param, "--webservicename", webservicename_param],
    compute_target=compute_target,
    source_directory="deployment",
    runconfig=run_config,
    allow_reuse=True
    )

In [19]:
validate_deployment_step = PythonScriptStep(
    name="Validate Deployment",
    script_name="validate.py",
    arguments=["--webservicename", webservicename_param],
    compute_target=compute_target,
    source_directory="deployment",
    runconfig=run_config,
    allow_reuse=False
    )

In [20]:
steps = [
    update_dataset_step,
    first_prepro_step,  
    second_prepro_step,
    estimator_step,
    deploy_step,
    validate_deployment_step
    ]

first_prepro_step.run_after(update_dataset_step)
deploy_step.run_after(estimator_step)
validate_deployment_step.run_after(deploy_step)

In [21]:
pipeline = Pipeline(workspace=ws, steps=[steps])

pipeline_run = experiment.submit(pipeline, pipeline_parameters={"threshold": 180.})

pipeline_run.wait_for_completion()

Created step Update Dataset [a1214e14][13bfb4a5-cf21-4ab1-9706-fbabde716681], (This step will run and generate new outputs)
Created step Parse dataset [f7c7b2a3][ca8bc6fe-7762-43ca-a520-9805d75fa22d], (This step is eligible to reuse a previous run's output)
Created step Convert to SKTime format [e91c8480][d6f4abf1-21ae-4f18-a7e9-66ffa0ae05d2], (This step is eligible to reuse a previous run's output)
Created step Train Model [251686e8][10e7ff64-1c2e-469b-821e-817421010b28], (This step is eligible to reuse a previous run's output)
Created step Deploy Model [811ebea6][de41a59f-bbe6-4e42-9a4b-25dd7fe9f893], (This step will run and generate new outputs)
Created step Validate Deployment [1dcb5fc3][bf37c6ac-ac78-4f11-aa04-1a2dc230db68], (This step will run and generate new outputs)
Submitted PipelineRun 25ae40f3-a411-4194-b19f-810619f7c5fb
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/MaxFreezerTemperatureExceededPipeline/runs/25ae40f3-a411-4194-b19f-810619f7c5fb?wsi

In [ ]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Freezer_SKTIME_pipeline",
    description="Freezer SKTIME train and deploy pipeline",
    version="2.0"
    )